In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor

from scipy.interpolate import make_interp_spline
from scipy.stats import ks_2samp
from scipy.spatial import ConvexHull

/tmp/ipykernel_189297/1551913226.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [3]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/output_repar_03-01/'               # Harmon server
name_project

'project_repartition_v4.0/output_repar_v9.2_01-01/'

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v4.0/output_repar_v9.2_01-01/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [6]:
name_project_probes = info_basic['name_project_probes']
name_project_targets = info_basic['name_project_targets']
dir_project_probes = os.path.join(dir_project_workspace, name_project_probes)
dir_project_targets = os.path.join(dir_project_workspace, name_project_targets)
print('dir_project_probes: ', dir_project_probes)
print('dir_project_targets: ', dir_project_targets)

dir_project_probes:  ./project/output_FJSJ_17-01/
dir_project_targets:  ./project/output_FJSJ_17-02/


In [7]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [8]:
key_subworks = info_basic['key_subworks']
dir_partition = dir_project + info_basic['dir_partition']

In [9]:
stations = {}
lat = {}
lon = {}
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations[key] = stations_this
    lat[key] = lat_stations_this.astype(float)
    lon[key] = lon_stations_this.astype(float)

# Be careful to delete!

In [10]:
#nums_delete = [24,73,90,96,119,137,138,141,190,215,216,219,221,230,234,237,246,296,297,312,321,333,347,371,421,439,441,443,450,467,478,479,488,498,499,515,516,525,526,541,542,543,547,550,552,554,650,651,652,573,574,588,589,594]
#nums_delete = [87,139,447,449,487,578]
#nums_delete = [32,74,128,131,165,308,350,377,378,380,394,395,396,397,432,451,459,480,481,484,518,529,557,558,559,579,584,586,590]
#nums_delete = [47,62]
#nums_delete =[61,64,250,268,269,482,560,561,562]
#nums_delete = [61,29,27,82,92,94,122,123,130,142,292,305,306,348,393,546]
nums_delete = [16,166,173,174,288,413,416,486,500,553,563,565]

In [11]:
key_subworks = []
for num in nums_delete:
    tag = str(num)
    for key_subwork in info_basic['key_subworks']:
        if tag == key_subwork.split('--')[0]:
            key_subworks.append(key_subwork)
key_subworks

['16--24-03',
 '166--42-07',
 '173--49-07',
 '174--50-07',
 '288--37-10',
 '413--32-13',
 '416--35-13',
 '486--22-15',
 '500--45-15',
 '553--23-17',
 '563--46-17',
 '565--48-17']

### Renew information

In [12]:
probe_accept_origin = info_basic_bi['probe_accept']
key_probe = info_basic_bi['probe']

In [13]:

for key_subwork in key_subworks:
    target = key_subwork.split('--')[0]
    if target in probe_accept_origin.keys():
        del probe_accept_origin[target]
    if target in key_probe.keys():
        del key_probe[key_subwork]
info_basic_bi['probe_accept'] = probe_accept_origin
info_basic_bi['probe'] = key_probe

In [14]:
for key in key_subworks:
    filePath = dir_partition + key + '.txt'
    if os.path.exists(filePath):
        os.remove(filePath)

In [15]:
for key in key_subworks:
    del info_basic['key_subworks'][info_basic['key_subworks'].index(key)]

In [16]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)
np.save(dir_project + 'Basic_info.npy', info_basic_bi)

### Delete stack and ds

In [17]:
dir_stack = dir_project + info_basic['dir_stack']
dir_ds = dir_project + info_basic['dir_ds']

In [18]:
list_ds = os.listdir(dir_ds)
#list_stack = os.listdir(dir_stack)

In [19]:
for key in key_subworks:
    stack_this = key + '_gather_linear.h5'
    stack_timewindow = key + '_gather_timewindow.h5'
    ds_this = 'ds_'+key +'.h5'
    if ds_this in list_ds:
        print('delete ', ds_this)
        os.remove(dir_ds + ds_this)
    #if stack_this in list_stack:
    #    print('delete ', stack_this)
    #    os.remove(dir_stack + stack_this)
    #if stack_timewindow in list_stack:
    #    print('delete ', stack_timewindow)
    #    os.remove(dir_stack + stack_timewindow)

delete  ds_16--24-03.h5
delete  ds_166--42-07.h5
delete  ds_173--49-07.h5
delete  ds_174--50-07.h5
delete  ds_288--37-10.h5
delete  ds_413--32-13.h5
delete  ds_416--35-13.h5
delete  ds_486--22-15.h5
delete  ds_500--45-15.h5
delete  ds_553--23-17.h5
delete  ds_563--46-17.h5
delete  ds_565--48-17.h5
